# **04 School Matcher**

In [ ]:
# Importing required packages
import os
import glob
import pandas as pd
pd.set_option('max_colwidth', 400)  #wider pandas columns upon render
import numpy as np
from itertools import islice
import thefuzz
from thefuzz import fuzz, process
from datetime import date

# Printing the working directory
os.getcwd()

## **Replacement of fuzzy string matches**

In [ ]:
# Importing merged star programs (merged with the applicant-level data from R)
programs = pd.read_csv("tables/STAR_Programs_2017_2023_merged.csv")
programs.head()

/tmp/ipykernel_6973/640562799.py:2: DtypeWarning: Columns (6,8) have mixed types. Specify dtype option on import or set low_memory=False.
  programs = pd.read_csv("tables/STAR_Programs_2017_2023_merged.csv")


,STAR.ID,Program.Institution,ACGME.Code,Interview.Offer,Matched,Geographic.Connection,Geographic.Preference.ERAS.Supp.App,Away.Rotation,Signaled.Program,Specialty,degrees,program_specialty
0,2017010001,Baylor College of Medicine,NaN,Y,Y,N,NaN,N,NaN,Anesthesiology,MD,Baylor College of Medicine_Anesthesiology
1,2017010001,Beth Israel Deaconess Medical Center/Harvard Medical School,NaN,N,N,N,NaN,N,NaN,Anesthesiology,MD,Beth Israel Deaconess Medical Center/Harvard Medical School_Anesthesiology
2,2017010001,Duke University Hospital,NaN,N,N,N,NaN,N,NaN,Anesthesiology,MD,Duke University Hospital_Anesthesiology
3,2017010001,Massachusetts General Hospital/Harvard Medical School,NaN,N,N,N,NaN,N,NaN,Anesthesiology,MD,Massachusetts General Hospital/Harvard Medical School_Anesthesiology
4,2017010001,Texas A&M College of Medicine-Scott and White,NaN,Y,N,N,NaN,N,NaN,Anesthesiology,MD,Texas A&M College of Medicine-Scott and White_Anesthesiology


In [ ]:
# Importing all the matched programs, mapping original program names to doximity names from 03_school_matcher.py
matching_docs = glob.glob("dox_matching/*.xlsx")
match_list = [pd.read_excel(x) for x in matching_docs]
match_df = pd.concat(match_list)
match_df["dox_program"] = match_df["dox_program"].replace({"USC/LAC+USC Medical Center": "University of Southern California/Los Angeles General Medical Center (USC/LA General)"})
match_df.head()

,ratio,partial_ratio,sort_ratio,set_ratio,partial_token,dox_program,star_program,specialty,mean_ratio
0,84.0,100.0,87.0,100.0,74.0,Baylor College of Medicine,Baylor College of Medicine (Houston),Plastic Surgery,89
1,NaN,NaN,NaN,NaN,NaN,Beaumont Health (Farmington Hills and Royal Oak),Beaumont Health (Farmington Hills and Royal Oak),Plastic Surgery,unable to match
2,76.0,100.0,76.0,100.0,78.0,Beth Israel Deaconess Medical Center,Beth Israel Deaconess Medical Center/Harvard Medical School,Plastic Surgery,86
3,83.0,100.0,83.0,100.0,85.0,Mass General Brigham/Brigham and Women's Hospital/Harvard Medical School,Brigham and Women's Hospital/Harvard Medical School,Plastic Surgery,90.2
4,42.0,100.0,43.0,100.0,81.0,Brown University,Brown University (Women and Infants Hospital of Rhode Island),Plastic Surgery,73.2


In [ ]:
# Looping through each specialty and replacing original program names with doximity program names for accurate rank list comparisons
programs_df_list = []
specialties = match_df["specialty"].sort_values().unique()  #unique specialties

for s in specialties:
    print("Matching {}".format(s))
    match_df_filt = match_df[match_df["specialty"]==s]
    programs_filt = programs[programs["Specialty"]==s].copy()
    
    # Making a replacement dictionary for each specialty
    specialty_dict = {}
    for index, row in match_df_filt.iterrows():
        dox_p = row["dox_program"]
        star_p = row["star_program"]
        specialty_dict[star_p] = dox_p
    
    # Replacing program institution names with doximity ones
    programs_filt.loc[:, "Program.Institution.Dox"] = programs_filt["Program.Institution"].replace(specialty_dict)
    programs_df_list.append(programs_filt)

# Concatenating values
programs_final = pd.concat(programs_df_list)
programs_final

Matching Anesthesiology
Matching Child Neurology
Matching Dermatology
Matching Emergency Medicine
Matching Family Medicine
Matching Internal Medicine
Matching Internal Medicine-Emergency Medicine
Matching Internal Medicine-Pediatrics
Matching Internal Medicine-Preliminary
Matching Internal Medicine-Psychiatry
Matching Neurodevelopmental Disabilities
Matching Neurological Surgery
Matching Neurology
Matching Obstetrics and Gynecology
Matching Ophthalmology
Matching Orthopaedic Surgery
Matching Osteopathic Neuromusculoskeletal Medicine
Matching Otolaryngology
Matching Pathology
Matching Pediatrics
Matching Pediatrics-Medical Genetics
Matching Pediatrics-Psychiatry-Child and Adolescent Psychiatry
Matching Physical Medicine and Rehabilitation
Matching Plastic Surgery
Matching Preventive Medicine
Matching Psychiatry
Matching Radiation Oncology
Matching Radiology-Diagnostic
Matching Radiology-Interventional
Matching Surgery
Matching Surgery-Preliminary
Matching Thoracic Surgery
Matching Trans

,STAR.ID,Program.Institution,ACGME.Code,Interview.Offer,Matched,Geographic.Connection,Geographic.Preference.ERAS.Supp.App,Away.Rotation,Signaled.Program,Specialty,degrees,program_specialty,Program.Institution.Dox
0,2017010001,Baylor College of Medicine,NaN,Y,Y,N,NaN,N,NaN,Anesthesiology,MD,Baylor College of Medicine_Anesthesiology,Baylor College of Medicine
1,2017010001,Beth Israel Deaconess Medical Center/Harvard Medical School,NaN,N,N,N,NaN,N,NaN,Anesthesiology,MD,Beth Israel Deaconess Medical Center/Harvard Medical School_Anesthesiology,Beth Israel Deaconess Medical Center
2,2017010001,Duke University Hospital,NaN,N,N,N,NaN,N,NaN,Anesthesiology,MD,Duke University Hospital_Anesthesiology,Duke University Hospital
3,2017010001,Massachusetts General Hospital/Harvard Medical School,NaN,N,N,N,NaN,N,NaN,Anesthesiology,MD,Massachusetts General Hospital/Harvard Medical School_Anesthesiology,Mass General Brigham/Massachusetts General Hospital
4,2017010001,Texas A&M College of Medicine-Scott and White,NaN,Y,N,N,NaN,N,NaN,Anesthesiology,MD,Texas A&M College of Medicine-Scott and White_Anesthesiology,Texas A&M College of Medicine-Scott and White Medical Center (Temple)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1634198,2023355034,USC/LAC+USC Medical Center,4.510512e+09,N,N,N,N,N,N,Vascular Surgery,MD,USC/LAC+USC Medical Center_Vascular Surgery,University of Southern California/Los Angeles General Medical Center (USC/LA General)
1634199,2023355034,Washington University/B-JH/SLCH Consortium,4.512800e+09,N,N,N,N,N,N,Vascular Surgery,MD,Washington University/B-JH/SLCH Consortium_Vascular Surgery,Washington University/B-JH/SLCH Consortium
1634200,2023355034,Yale-New Haven Medical Center,4.510821e+09,N,N,N,N,N,N,Vascular Surgery,MD,Yale-New Haven Medical Center_Vascular Surgery,Yale-New Haven Medical Center
1634201,2023355034,Zucker School of Medicine at Hofstra/Northwell,4.513500e+09,N,N,N,N,N,N,Vascular Surgery,MD,Zucker School of Medicine at Hofstra/Northwell_Vascular Surgery,Zucker School of Medicine at Hofstra/Northwell


In [ ]:
# Exporting the dataframe
#programs_final.to_csv("tables/STAR_Programs_2017_2023_final_matched.csv", index=False)

## **Fuzzy string match for inexact program matches**
#### Imported from R

#### Test code for 03_school_matcher.py

In [ ]:
star = pd.read_csv("tables/texasSTAR_unique_programs.csv")
unmatched = pd.read_csv("tables/unmatched_unique_programs.csv")
dox = pd.read_excel("tables/doximity_rankings.xlsx")

display(star.head())
display(unmatched.head())
display(dox.head())

,star_programs,program,specialty
0,Baylor College of Medicine_Anesthesiology,Baylor College of Medicine,Anesthesiology
1,Beth Israel Deaconess Medical Center/Harvard Medical School_Anesthesiology,Beth Israel Deaconess Medical Center/Harvard Medical School,Anesthesiology
2,Duke University Hospital_Anesthesiology,Duke University Hospital,Anesthesiology
3,Massachusetts General Hospital/Harvard Medical School_Anesthesiology,Massachusetts General Hospital/Harvard Medical School,Anesthesiology
4,Texas A&M College of Medicine-Scott and White_Anesthesiology,Texas A&M College of Medicine-Scott and White,Anesthesiology


,unmatched_programs,program,specialty
0,Beth Israel Deaconess Medical Center/Harvard Medical School_Anesthesiology,Beth Israel Deaconess Medical Center/Harvard Medical School,Anesthesiology
1,Massachusetts General Hospital/Harvard Medical School_Anesthesiology,Massachusetts General Hospital/Harvard Medical School,Anesthesiology
2,Texas A&M College of Medicine-Scott and White_Anesthesiology,Texas A&M College of Medicine-Scott and White,Anesthesiology
3,University of Texas Health Science Center School of Medicine at San Antonio_Anesthesiology,University of Texas Health Science Center School of Medicine at San Antonio,Anesthesiology
4,University of Texas Southwestern Medical School_Anesthesiology,University of Texas Southwestern Medical School,Anesthesiology


,program,location,program_size,rank,specialty,sort_by
0,Mass General Brigham/Massachusetts General Hospital,Boston,116,1,Anesthesiology,reputation
1,University of California (San Francisco),San Francisco,96,2,Anesthesiology,reputation
2,Stanford Health Care-Sponsored Stanford University,Stanford,108,3,Anesthesiology,reputation
3,Mass General Brigham/Brigham and Women's Hospital,Boston,128,4,Anesthesiology,reputation
4,Duke University Hospital,Durham,60,5,Anesthesiology,reputation


In [ ]:
for index, row in islice(unmatched.iterrows(), 10):
    program = row["program"]
    specialty = row["specialty"]
    print(program, specialty)

Beth Israel Deaconess Medical Center/Harvard Medical School Anesthesiology
Massachusetts General Hospital/Harvard Medical School Anesthesiology
Texas A&M College of Medicine-Scott and White Anesthesiology
University of Texas Health Science Center School of Medicine at San Antonio Anesthesiology
University of Texas Southwestern Medical School Anesthesiology
Brigham and Women's Hospital/Harvard Medical School Anesthesiology
Icahn School of Medicine at Mount Sinai Anesthesiology
Icahn School of Medicine at Mount Sinai (St Luke's-Roosevelt Hospital Center) Anesthesiology
Loyola University Anesthesiology
New York University School of Medicine Anesthesiology


In [ ]:
match_list = []
for index, row in islice(unmatched.iterrows(), 5):
    program = row["program"]
    specialty = row["specialty"]
    
    dox_filt = dox[dox["specialty"]==specialty]
    dox_filt = dox_filt["program"].unique()
    
    ratio = []
    partial_ratio = []
    sort_ratio = []
    set_ratio = []
    partial_token = []
    dox_program_list = []
    for dox_program in dox_filt:
        ratio.append(fuzz.ratio(program, dox_program))
        partial_ratio.append(fuzz.partial_ratio(program, dox_program))
        sort_ratio.append(fuzz.token_sort_ratio(program, dox_program))
        set_ratio.append(fuzz.token_set_ratio(program, dox_program))
        partial_token.append(fuzz.partial_token_sort_ratio(program, dox_program))
        dox_program_list.append(dox_program)
    
    df = pd.DataFrame({"ratio": ratio, "partial_ratio": partial_ratio, "sort_ratio": sort_ratio, "set_ratio": set_ratio, "partial_token": partial_token, "dox_program": dox_program_list})
    df["star_program"] = program
    df["specialty"] = specialty
    df["mean_ratio"] = df.mean(axis=1)
    df = df.sort_values("mean_ratio", ascending=False).reset_index(drop=True)
    display(df.head(10))
    
    selection = input("Select the top match by index: ")
    if selection=="done":
        print("finished with looping")
        break
    try:
        selection = int(selection)
        matched_df = pd.DataFrame(df.iloc[selection]).transpose()
        match_list.append(matched_df)
        display(matched_df)
    except:
        print("unable to match")
        unmatched_df = pd.DataFrame({"ratio": np.nan, "partial_ratio": np.nan, "sort_ratio": np.nan, "set_ratio": np.nan, "partial_token": np.nan, "dox_program": program, "star_program": program, "specialty": specialty, "mean_ratio": "unable to match"}, index=[0])
        match_list.append(unmatched_df)
        display(unmatched_df)

df_total = pd.concat(match_list)
display(df_total)

C:\Users\TooFastDan\AppData\Local\Temp\ipykernel_19540\696497551.py:26: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df["mean_ratio"] = df.mean(axis=1)


,ratio,partial_ratio,sort_ratio,set_ratio,partial_token,dox_program,star_program,specialty,mean_ratio
0,76,100,76,100,78,Beth Israel Deaconess Medical Center,Beth Israel Deaconess Medical Center/Harvard Medical School,Anesthesiology,86.0
1,50,75,43,72,79,UMass Chan Medical School,Beth Israel Deaconess Medical Center/Harvard Medical School,Anesthesiology,63.8
2,46,85,46,82,60,Maine Medical Center,Beth Israel Deaconess Medical Center/Harvard Medical School,Anesthesiology,63.8
3,43,80,41,82,60,Tufts Medical Center,Beth Israel Deaconess Medical Center/Harvard Medical School,Anesthesiology,61.2
4,48,80,48,72,57,Maimonides Medical Center,Beth Israel Deaconess Medical Center/Harvard Medical School,Anesthesiology,61.0
5,45,76,40,80,61,Albany Medical Center,Beth Israel Deaconess Medical Center/Harvard Medical School,Anesthesiology,60.4
6,47,73,45,70,59,Westchester Medical Center,Beth Israel Deaconess Medical Center/Harvard Medical School,Anesthesiology,58.8
7,47,73,49,70,54,St Joseph's Medical Center,Beth Israel Deaconess Medical Center/Harvard Medical School,Anesthesiology,58.6
8,51,64,51,62,60,Rutgers Health/New Jersey Medical School,Beth Israel Deaconess Medical Center/Harvard Medical School,Anesthesiology,57.6
9,45,69,52,65,55,Yale-New Haven Medical Center,Beth Israel Deaconess Medical Center/Harvard Medical School,Anesthesiology,57.2


,ratio,partial_ratio,sort_ratio,set_ratio,partial_token,dox_program,star_program,specialty,mean_ratio
0,76,100,76,100,78,Beth Israel Deaconess Medical Center,Beth Israel Deaconess Medical Center/Harvard Medical School,Anesthesiology,86.0


C:\Users\TooFastDan\AppData\Local\Temp\ipykernel_19540\696497551.py:26: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df["mean_ratio"] = df.mean(axis=1)


,ratio,partial_ratio,sort_ratio,set_ratio,partial_token,dox_program,star_program,specialty,mean_ratio
0,58,74,63,82,74,Mass General Brigham/Massachusetts General Hospital,Massachusetts General Hospital/Harvard Medical School,Anesthesiology,70.2
1,54,75,44,72,75,UMass Chan Medical School,Massachusetts General Hospital/Harvard Medical School,Anesthesiology,64.0
2,54,61,47,54,61,Rutgers Health/New Jersey Medical School,Massachusetts General Hospital/Harvard Medical School,Anesthesiology,55.4
3,47,53,51,62,58,Mass General Brigham/Brigham and Women's Hospital,Massachusetts General Hospital/Harvard Medical School,Anesthesiology,54.2
4,51,56,45,53,54,Rutgers Health/Robert Wood Johnson Medical School,Massachusetts General Hospital/Harvard Medical School,Anesthesiology,51.8
5,48,51,45,54,58,Case Western Reserve University/University Hospitals Cleveland Medical Center,Massachusetts General Hospital/Harvard Medical School,Anesthesiology,51.2
6,33,63,36,59,63,Henry Ford Hospital,Massachusetts General Hospital/Harvard Medical School,Anesthesiology,50.8
7,37,64,37,53,59,Robert Packer Hospital,Massachusetts General Hospital/Harvard Medical School,Anesthesiology,50.0
8,39,62,39,50,58,Duke University Hospital,Massachusetts General Hospital/Harvard Medical School,Anesthesiology,49.6
9,40,46,48,48,65,Methodist Hospital (Houston),Massachusetts General Hospital/Harvard Medical School,Anesthesiology,49.4


...

,ratio,partial_ratio,sort_ratio,set_ratio,partial_token,dox_program,star_program,specialty,mean_ratio
0,69,83,83,98,79,University of Texas Health Science Center San Antonio Joe and Teresa Lozano Long School of Medicine,University of Texas Health Science Center School of Medicine at San Antonio,Anesthesiology,82.4


C:\Users\TooFastDan\AppData\Local\Temp\ipykernel_19540\696497551.py:26: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df["mean_ratio"] = df.mean(axis=1)


,ratio,partial_ratio,sort_ratio,set_ratio,partial_token,dox_program,star_program,specialty,mean_ratio
0,87,93,85,92,92,University of Texas Southwestern Medical Center,University of Texas Southwestern Medical School,Anesthesiology,89.8
1,57,89,51,79,76,University of Toledo,University of Texas Southwestern Medical School,Anesthesiology,70.4
2,60,77,60,72,70,University of Rochester,University of Texas Southwestern Medical School,Anesthesiology,67.8
3,70,68,59,76,61,University of Texas Medical Branch Hospitals,University of Texas Southwestern Medical School,Anesthesiology,66.8
4,58,78,53,72,73,UMass Chan Medical School,University of Texas Southwestern Medical School,Anesthesiology,66.8
5,56,80,50,76,71,University of Chicago,University of Texas Southwestern Medical School,Anesthesiology,66.6
6,53,80,47,76,71,University of Florida,University of Texas Southwestern Medical School,Anesthesiology,65.4
7,52,78,52,74,69,University of Michigan,University of Texas Southwestern Medical School,Anesthesiology,65.0
8,52,78,49,74,69,University of Colorado,University of Texas Southwestern Medical School,Anesthesiology,64.4
9,52,78,49,74,69,University of Maryland,University of Texas Southwestern Medical School,Anesthesiology,64.4


,ratio,partial_ratio,sort_ratio,set_ratio,partial_token,dox_program,star_program,specialty,mean_ratio
1,57,89,51,79,76,University of Toledo,University of Texas Southwestern Medical School,Anesthesiology,70.4


,ratio,partial_ratio,sort_ratio,set_ratio,partial_token,dox_program,star_program,specialty,mean_ratio
0,76,100,76,100,78,Beth Israel Deaconess Medical Center,Beth Israel Deaconess Medical Center/Harvard Medical School,Anesthesiology,86.0
0,58,74,63,82,74,Mass General Brigham/Massachusetts General Hospital,Massachusetts General Hospital/Harvard Medical School,Anesthesiology,70.2
0,NaN,NaN,NaN,NaN,NaN,Texas A&M College of Medicine-Scott and White,Texas A&M College of Medicine-Scott and White,Anesthesiology,unable to match
0,69,83,83,98,79,University of Texas Health Science Center San Antonio Joe and Teresa Lozano Long School of Medicine,University of Texas Health Science Center School of Medicine at San Antonio,Anesthesiology,82.4
1,57,89,51,79,76,University of Toledo,University of Texas Southwestern Medical School,Anesthesiology,70.4


In [ ]:
ratio = []
for index, row in star.iterrows():
    r = fuzz.ratio(row["star_programs"], row["program"])
    ratio.append(r)
star["ratio"] = ratio
star

,star_programs,program,specialty,ratio
0,Baylor College of Medicine_Anesthesiology,Baylor College of Medicine,Anesthesiology,78
1,Beth Israel Deaconess Medical Center/Harvard M...,Beth Israel Deaconess Medical Center/Harvard M...,Anesthesiology,89
2,Duke University Hospital_Anesthesiology,Duke University Hospital,Anesthesiology,76
3,Massachusetts General Hospital/Harvard Medical...,Massachusetts General Hospital/Harvard Medical...,Anesthesiology,88
4,Texas A&M College of Medicine-Scott and White_...,Texas A&M College of Medicine-Scott and White,Anesthesiology,86
...,...,...,...,...
10729,Loma Linda University Health Education Consort...,Loma Linda University Health Education Consort...,Preventive Medicine,90
10730,Texas Department of State Health Services Publ...,Texas Department of State Health Services Publ...,Preventive Medicine,90
10731,University at Buffalo Public Health and Genera...,University at Buffalo Public Health and Genera...,Preventive Medicine,87
10732,University of Texas Health Science Center at H...,University of Texas Health Science Center at H...,Preventive Medicine,88


In [ ]:
today = str(date.today())
print(today + "_Anesthiology")

2024-12-12_Anesthiology
